VS shortcuts to remember:
- CTRL + SPACE when importing functions from module to list them all
- F12 to open function init.py
- ALT + F12 to peek at function init.py
- Mouse hover to read function description (+ CTRL)

## PuLP Linear Programming intro

Linear Programming for optimization tasks in Python

<img src="https://i.ytimg.com/vi/cXHvC_FGx24/maxresdefault.jpg" width="700"/>


In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Basic equation optimization

Parts
- Objective function (min, max, match)
- Constraints (variable bounds, **linear** combination of variables)

Modeling attributes:
- Solver type
- Initial guesses (warm-start)
- Objective & variable types (floats / ints / binaries)


Problem: production of 2 products (A, B), using raw materials in optimal way to maximize profit

<img src="src/pulp_intro_basic_task.png" width="700"/>

In [11]:
import pulp

Define objective and variables

In [12]:
prob = pulp.LpProblem(name = 'Maximize profit', sense = pulp.const.LpMaximize)

a = pulp.LpVariable('A', lowBound = 0, cat = pulp.const.LpContinuous)
b = pulp.LpVariable('B', lowBound = 0, cat = pulp.const.LpContinuous)

prob += 30*a + 40*b

Set constraints

In [13]:
prob += 12*a + 4*b <= 300, 'Raw material I'
prob += 4*a + 4*b <= 120, 'Raw material II'
prob += 3*a + 12*b <= 252, 'Raw material III'

Check LP problem

In [14]:
print(prob)

Maximize_profit:
MAXIMIZE
30*A + 40*B + 0
SUBJECT TO
Raw_material_I: 12 A + 4 B <= 300

Raw_material_II: 4 A + 4 B <= 120

Raw_material_III: 3 A + 12 B <= 252

VARIABLES
A Continuous
B Continuous



Select solver, solve, check status and results

In [15]:
print('Possible solvers:', pulp.listSolvers())
print('Possible status values:', pulp.LpStatus)

Possible solvers: ['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'PULP_CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD']
Possible status values: {0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'}


In [16]:
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [17]:
print('Used solver:', prob.solver)

Used solver: <pulp.apis.coin_api.PULP_CBC_CMD object at 0x0000022E7ECE30D0>


In [18]:
for variable in prob.variables():
    print("{} = {}".format(variable.name, variable.varValue))

A = 12.0
B = 18.0


In [19]:
OPTIMUM = pulp.value(prob.objective)
print('Optimal profit:', OPTIMUM)

Optimal profit: 1080.0


### Advanced example

When there're many constraints and variables (most likely scenario in real life) we don't have to set them 1-by-1 manually. We can (and do) use list comprehension!

**Problem**: Cost optimization of cat food with respect to it's ingredients

Costs per gram:
- chicken: $0.013
- beef: $0.008
- mutton: $0.01
- rice: $0.002
- wheat: $0.005
- gel: $0.001

Ingredient contributions:

<img src="data/pulp_intro_cat_food_ingredients.png" width="500"/>

Ingredient requirements
- Protein: min 8g/100g
- Fat: min 6g/100g
- Fibre: max 4g/100g
- Salt: max 0.4g/100g

In [20]:
ingredients = pd.DataFrame({'Stuff' : ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat', 'Gel'],
                            'Cost' : [0.013, 0.008, 0.01, 0.002, 0.005, 0.001],
                            'Protein' : [0.1, 0.2, 0.15, 0, 0.04, 0],
                            'Fat' : [0.08, 0.1, 0.11, 0.01, 0.01, 0],
                            'Fibre' : [0.001, 0.005, 0.003, 0.1, 0.15, 0],
                            'Salt' : [0.002, 0.005, 0.007, 0.002, 0.008, 0]})

ingredients

,Stuff,Cost,Protein,Fat,Fibre,Salt
0,Chicken,0.013,0.10,0.08,0.001,0.002
1,Beef,0.008,0.20,0.10,0.005,0.005
2,Mutton,0.010,0.15,0.11,0.003,0.007
3,Rice,0.002,0.00,0.01,0.100,0.002
4,Wheat,0.005,0.04,0.01,0.150,0.008
5,Gel,0.001,0.00,0.00,0.000,0.000


Define problem and objective

In [21]:
prob = pulp.LpProblem(name = 'Cat food cost minimization', sense = pulp.const.LpMinimize)

Constraint dicts

In [22]:
_ingredients = ingredients['Stuff'].tolist()

_costs = dict(zip(ingredients['Stuff'], ingredients['Cost']))

_protein = dict(zip(ingredients['Stuff'], ingredients['Protein']))
_fat = dict(zip(ingredients['Stuff'], ingredients['Fat']))
_fibre = dict(zip(ingredients['Stuff'], ingredients['Fibre']))
_salt = dict(zip(ingredients['Stuff'], ingredients['Salt']))

Create dictionary of variables for LP model

In [23]:
_variables = pulp.LpVariable.dicts( 'ingredient', _ingredients, lowBound = 0, upBound = 1)
_variables

{'Chicken': ingredient_Chicken,
 'Beef': ingredient_Beef,
 'Mutton': ingredient_Mutton,
 'Rice': ingredient_Rice,
 'Wheat': ingredient_Wheat,
 'Gel': ingredient_Gel}

Define function to minimize

In [24]:
prob += pulp.lpSum([_variables[i] * _costs[i] for i in _ingredients])

In [25]:
prob

Cat_food_cost_minimization:
MINIMIZE
0.008*ingredient_Beef + 0.013*ingredient_Chicken + 0.001*ingredient_Gel + 0.01*ingredient_Mutton + 0.002*ingredient_Rice + 0.005*ingredient_Wheat + 0.0
VARIABLES
ingredient_Beef <= 1 Continuous
ingredient_Chicken <= 1 Continuous
ingredient_Gel <= 1 Continuous
ingredient_Mutton <= 1 Continuous
ingredient_Rice <= 1 Continuous
ingredient_Wheat <= 1 Continuous

Add constraints

In [26]:
# Ratio of ingredients has to sum up to 1
prob += pulp.lpSum(_variables[i] for i in _ingredients) == 1, 'Ratios add up to 1'

# Protein, fat, fibre & salt fulfill their requirements
prob += pulp.lpSum(_variables[i]  * _protein[i] for i in _ingredients) >= 0.08, 'Protein at least 8g'
prob += pulp.lpSum(_variables[i]  * _fat[i] for i in _ingredients) >= 0.06, 'Fat at least 6g'
prob += pulp.lpSum(_variables[i]  * _fibre[i] for i in _ingredients) <= 0.02, 'Fiber at most 2g'
prob += pulp.lpSum(_variables[i]  * _salt[i] for i in _ingredients) <= 0.004, 'Salt at most 0.4g'

In [27]:
print(prob)

Cat_food_cost_minimization:
MINIMIZE
0.008*ingredient_Beef + 0.013*ingredient_Chicken + 0.001*ingredient_Gel + 0.01*ingredient_Mutton + 0.002*ingredient_Rice + 0.005*ingredient_Wheat + 0.0
SUBJECT TO
Ratios_add_up_to_1: ingredient_Beef + ingredient_Chicken + ingredient_Gel
 + ingredient_Mutton + ingredient_Rice + ingredient_Wheat = 1

Protein_at_least_8g: 0.2 ingredient_Beef + 0.1 ingredient_Chicken
 + 0.15 ingredient_Mutton + 0.04 ingredient_Wheat >= 0.08

Fat_at_least_6g: 0.1 ingredient_Beef + 0.08 ingredient_Chicken
 + 0.11 ingredient_Mutton + 0.01 ingredient_Rice + 0.01 ingredient_Wheat
 >= 0.06

Fiber_at_most_2g: 0.005 ingredient_Beef + 0.001 ingredient_Chicken
 + 0.003 ingredient_Mutton + 0.1 ingredient_Rice + 0.15 ingredient_Wheat
 <= 0.02

Salt_at_most_0.4g: 0.005 ingredient_Beef + 0.002 ingredient_Chicken
 + 0.007 ingredient_Mutton + 0.002 ingredient_Rice + 0.008 ingredient_Wheat
 <= 0.004

VARIABLES
ingredient_Beef <= 1 Continuous
ingredient_Chicken <= 1 Continuous
ingredient

Run optimization & check optimum

In [28]:
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [29]:
for variable in prob.variables():
    print("{} = {}".format(variable.name, variable.varValue))

ingredient_Beef = 0.6
ingredient_Chicken = 0.0
ingredient_Gel = 0.4
ingredient_Mutton = 0.0
ingredient_Rice = 0.0
ingredient_Wheat = 0.0


In [30]:
OPTIMUM = pulp.value(prob.objective)
print('Optimal cost per 100g ($):', OPTIMUM)

Optimal cost per 100g ($): 0.0052


#### Just beef and gel? Let's set some minimum constraints for chicken and rice, and maximum for gel

In [31]:
_min_constraint = {'Chicken': 0.1,
                    'Beef': 0.25,
                    'Mutton': 0,
                    'Rice': 0.1,
                    'Wheat': 0,
                    'Gel': 0}

_max_constraint = {'Chicken': 1,
                    'Beef': 1,
                    'Mutton': 1,
                    'Rice': 1,
                    'Wheat': 1,
                    'Gel': 0.15}

In [32]:
_variables = pulp.LpVariable.dicts( 'ingredient', _ingredients, lowBound = 0, upBound = 1)

for i in _variables.keys():
    _variables[i].lowBound = _min_constraint[i]
    _variables[i].upBound = _max_constraint[i]

In [33]:
_variables['Chicken'].to_dict()

{'lowBound': 0.1,
 'upBound': 1,
 'cat': 'Continuous',
 'varValue': None,
 'dj': None,
 'name': 'ingredient_Chicken'}

In [34]:
### WHole process

# Set variable bounds
_variables = pulp.LpVariable.dicts( 'ingredient', _ingredients, lowBound = 0, upBound = 1)

for i in _variables.keys():
    _variables[i].lowBound = _min_constraint[i]
    _variables[i].upBound = _max_constraint[i]

# Create LP problem
prob = pulp.LpProblem(name = 'Cat food cost minimization', sense = pulp.const.LpMinimize)

# Objective function
prob += pulp.lpSum([_variables[i] * _costs[i] for i in _ingredients])

# Ratio of ingredients has to sum up to 1
prob += pulp.lpSum(_variables[i] for i in _ingredients) == 1, 'Ratios add up to 1'

# Protein, fat, fibre & salt fulfill their requirements
prob += pulp.lpSum(_variables[i]  * _protein[i] for i in _ingredients) >= 0.08, 'Protein at least 8g'
prob += pulp.lpSum(_variables[i]  * _fat[i] for i in _ingredients) >= 0.06, 'Fat at least 6g'
prob += pulp.lpSum(_variables[i]  * _fibre[i] for i in _ingredients) <= 0.02, 'Fiber at most 2g'
prob += pulp.lpSum(_variables[i]  * _salt[i] for i in _ingredients) <= 0.004, 'Salt at most 0.4g'

status = prob.solve()
print('Status:', pulp.LpStatus[status])

for variable in prob.variables():
    print("{} = {}".format(variable.name, variable.varValue))

OPTIMUM = pulp.value(prob.objective)
print('Optimal cost per 100g ($):', OPTIMUM)


Status: Optimal
ingredient_Beef = 0.58
ingredient_Chicken = 0.1
ingredient_Gel = 0.15
ingredient_Mutton = 0.0
ingredient_Rice = 0.17
ingredient_Wheat = 0.0
Optimal cost per 100g ($): 0.00643


Instead of $0.0052 now the minimum cost is $0.00643

In [35]:
print(prob)

Cat_food_cost_minimization:
MINIMIZE
0.008*ingredient_Beef + 0.013*ingredient_Chicken + 0.001*ingredient_Gel + 0.01*ingredient_Mutton + 0.002*ingredient_Rice + 0.005*ingredient_Wheat + 0.0
SUBJECT TO
Ratios_add_up_to_1: ingredient_Beef + ingredient_Chicken + ingredient_Gel
 + ingredient_Mutton + ingredient_Rice + ingredient_Wheat = 1

Protein_at_least_8g: 0.2 ingredient_Beef + 0.1 ingredient_Chicken
 + 0.15 ingredient_Mutton + 0.04 ingredient_Wheat >= 0.08

Fat_at_least_6g: 0.1 ingredient_Beef + 0.08 ingredient_Chicken
 + 0.11 ingredient_Mutton + 0.01 ingredient_Rice + 0.01 ingredient_Wheat
 >= 0.06

Fiber_at_most_2g: 0.005 ingredient_Beef + 0.001 ingredient_Chicken
 + 0.003 ingredient_Mutton + 0.1 ingredient_Rice + 0.15 ingredient_Wheat
 <= 0.02

Salt_at_most_0.4g: 0.005 ingredient_Beef + 0.002 ingredient_Chicken
 + 0.007 ingredient_Mutton + 0.002 ingredient_Rice + 0.008 ingredient_Wheat
 <= 0.004

VARIABLES
0.25 <= ingredient_Beef <= 1 Continuous
0.1 <= ingredient_Chicken <= 1 Contin